# Logistic Regression Analysis

In [ ]:
"""
Install necessary packages
"""

# !pip install numpy
# !pip install matplotlib
# !pip install sklearn
# !pip install pandas

In [ ]:
"""
Import necessary packages
"""
import os
import glob 
import re

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches

import pandas as pd

from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import accuracy_score

"""
Imports from preprocessing codebase
"""
from preprocessing.center_finding import find_centroid
from models.feature_engineering import feature_9a_ratio
from models.feature_engineering import feature_5a_peak_location
from models.feature_engineering import feature_5a_9a_peak_location_ratio

In [ ]:
"""
Parameters
"""
h=256
w=256

In [ ]:
"""
Import data set
"""
# Set filepaths
filesdir = ""

timestamp = ""

output_style = "quad_folded"
# classifications = ["normal", "cancer"]
classification = "samples"

datalist = []
filenames = []

# Set directory
data_dir = "preprocessed_" + classification + "_" + timestamp
# print(classification.capitalize() + " input folder: " + data_dir)

input_dir = os.path.join(filesdir, data_dir, output_style)

print(input_dir)

# Get files list
input_filenames = glob.glob(os.path.join(input_dir,"*.txt"))
input_filenames.sort()

file_count = len(input_filenames)

print("Number of files: " + str(file_count))

# Store data
image_data = np.zeros((file_count, h, w),dtype=np.uint16)
for jdx, input_file in enumerate(input_filenames):
    image_data[jdx] = np.loadtxt(input_file,dtype=np.uint16)
        
datalist = image_data
filenames = input_filenames

In [ ]:
"""
Collect the features
"""

# data = np.concatenate((datalist[0],datalist[1]),dtype=np.uint16)
data = datalist
# files = filenames[0] + filenames[1]
files = filenames

X1 = np.zeros((len(data)))
X2 = np.zeros((len(data)))

print(data.shape)

for idx, image in enumerate(data[:]):
#     x1_peak_location, roi, roi_center, roi_anchor =  feature_5a_peak_location(image)
#     X1[idx] = x1_peak_location
    x1_peak_location_ratio = feature_5a_9a_peak_location_ratio(image)
    X1[idx] = x1_peak_location_ratio
    
    x2_intensity_ratio, rois, centers, anchors = feature_9a_ratio(image)
    X2[idx] = x2_intensity_ratio


    filename = os.path.basename(files[idx])
    
    visualize=False
    if visualize:
        fig = plt.figure(dpi=100)
        fig.set_size_inches(4*1,4*1)
        fig.set_facecolor("white")

        fig.suptitle(filename)
        ax = fig.add_subplot(1,1,1)

        # Plot image
        plt.imshow(image,cmap="gray")
        
        # Plot 5A window
        rect = patches.Rectangle((roi_anchor[1]-1, roi_anchor[0]-1), roi_anchor[3], roi_anchor[2],
                                     linewidth=1, edgecolor='b', facecolor='none')
        ax.add_patch(rect)

        # Plot 9A windows
        for anchor in anchors:
            # Note: xy axis is used for this part
            rect = patches.Rectangle((anchor[1]-1, anchor[0]-1), anchor[3], anchor[2],
                                     linewidth=1, edgecolor='r', facecolor='none')
            ax.add_patch(rect)
            
        plt.xticks(())
        plt.yticks(())

        plt.show()

In [ ]:
"""
Merge the data into one dataframe
"""
csv_file = os.path.join(filesdir, "samples.csv")
df1 = pd.read_csv(csv_file, sep=",")

fnames = [os.path.basename(fname) for fname in filenames]
barcodes = [re.search("A[0-9]+",fname)[0] for fname in fnames]

dataframe2_arr = np.array([barcodes, X1, X2]).T

df2 = pd.DataFrame(data=dataframe2_arr,
                   columns=["Barcode","5a_9a_peak_location_ratio","9a_intensity_ratio"])

df3 = pd.merge(df1, df2)
df = df3

In [ ]:
"""
Export features to csv
"""
export = True
if export:
    df.to_csv(os.path.join(filesdir, "updated_features.csv"))

In [ ]:
"""
Prep data and labels
"""
# Data
X = np.array([X1.T, X2.T]).T
print(X.shape)

"""
Average over samples
"""
csv_num = df["Patient"].nunique()

print("There are " + str(csv_num) + " unique samples.")

# Y = np.zeros((X.shape[0],1))
# Y = df['Cancer'].values.reshape((-1,1))
# print(Y.shape)

# Labels
Y = np.zeros((csv_num,1),dtype=bool)
X_new = np.zeros((csv_num,2))

# Loop over each sample
# and average X and label Y

for idx in np.arange(csv_num):
    # Get a sample
    sample = df.loc[df['Barcode'] == barcodes[idx]]
    patient = sample.values[0][1]
    # Get all specimens from the same patient
    df_rows = df.loc[df['Patient'] == patient]
    indices = df_rows.index
    # Now average across all samples
    X_new[idx,:] = np.mean(X[indices,:],axis=0)
    # Get the labels for the samples, first one is ok'
    Y[idx] = df_rows["Cancer"][indices[0]]


X = X_new
print("Total data count after averaging:")
print(Y.shape)

print("Normal data count:")
print(np.sum(Y == False))
print("Cancer data count:")
print(np.sum(Y == True))

In [ ]:
"""
Perform Logistic Regression
"""
# x-axis: feature 1
# y-axis: feature 2
# normal: o's
# cancer: x's

# Perform logistic regression
logreg = LogisticRegression(C=1e6,class_weight="balanced")
logreg.fit(X, Y)
print("Score: {:.2f}".format(logreg.score(X,Y)))

theta1, theta2 = logreg.coef_.ravel()
theta0 = logreg.intercept_[0]
theta = np.array([[theta0, theta1, theta2]])
print("Theta array:")
print(theta)

x1_test = np.linspace(np.min(X[:,0]),np.max(X[:,0]),10)
x2_test = -(theta1*x1_test+theta0)/theta2

# Plot
fig2 = plt.figure(1,dpi=100,figsize=(8, 8))
fig2.set_facecolor("white")

fig2.suptitle("Logistic Regression Analysis - April 21, May 4, 9, 2022")

# Plot linear decision boundary
plt.plot(x1_test, x2_test)


# Predict
Y_predict = theta1*X[:,0] + theta2*X[:,1] + theta0 > 0

# Get scores
precision = precision_score(Y, Y_predict)
print("Precision:")
print(precision)
recall = recall_score(Y, Y_predict)
print("Recall:")
print(recall)
# False positive rate: false positives
# Of samples identified as positive, what percentage are false
print("False positive rate:")
false_positives = np.sum(Y[Y_predict == True] == False)
predicted_positives = np.sum(Y_predict == True)
false_positive_rate = false_positives/predicted_positives
print(false_positive_rate)

# Accuracy = number of correct predictions / total predictions
# Balanced accuracy score, weights by counts
balanced_accuracy = balanced_accuracy_score(Y, Y_predict)
print("Balanced accuracy:")
print(balanced_accuracy)
# Unbalanced accuracy
unbalanced_accuracy = accuracy_score(Y, Y_predict)
print("Unbalanced accuracy:")
print(unbalanced_accuracy)


# Plot data
normal_indices = np.where(Y == False)
cancer_indices = np.where(Y == True)

# Normal
plt.scatter(X[normal_indices,0],X[normal_indices,1],color='b',marker='o',label="Normal")
# Cancer
plt.scatter(X[cancer_indices,0],X[cancer_indices,1],color='r',marker='x',label="Cancer")


# plt.xlabel("5A Peak Location")
plt.xlabel("5.1A / 9.8A Peak Location Ratio")
plt.ylabel("9.8A Intensity Ratio")

# Set plot limits
x_min, x_max = X[:, 0].min(), X[:, 0].max()
y_min, y_max = X[:, 1].min(), X[:, 1].max()

# plt.xlim([x_min-0.01, x_max+0.01])
# plt.ylim([y_min-0.01,y_max+0.01])

plt.legend()

plt.show()